In [88]:
!pip install tensorflow
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
!nvidia-smi

Fri Mar 17 17:55:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    33W /  70W |   2389MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
# Part 1 - Building the CNN
#importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
import h5py

#Input Image Size
sz=300

In [92]:
# Initialing the CNN
classifier = Sequential()

In [93]:
# Step 1 - Convolutio Layer 
classifier.add(Convolution2D(32, (3,3) , input_shape = (sz, sz,3), activation = 'relu'))

In [94]:
#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [95]:
# Adding second convolution layer
classifier.add(Convolution2D(32, (3,3) , activation = 'relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [96]:
#Adding 3rd Concolution Layer
classifier.add(Convolution2D(64, (3,3) , activation = 'relu'))
classifier.add(MaxPooling2D(pool_size =(2,2)))


In [97]:
#Step 3 - Flattening
classifier.add(Flatten())

In [98]:
#Step 4 - Full Connection
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(36, activation = 'softmax')) # 36 Units are the value of no of Classes

In [99]:
#Compiling The CNN
classifier.compile(
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [100]:
# Classifier Summary
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 149, 149, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 147, 147, 32)      9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 73, 73, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 35, 35, 64)      

In [101]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [102]:
#Dataset Store in Variable
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/train',
        target_size=(sz, sz),
        batch_size=32,
        # color_mode='grayscale',
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/test',
        target_size=(sz, sz),
        batch_size=32,
        # color_mode='grayscale',
        class_mode='categorical')

Found 7605 images belonging to 36 classes.
Found 3330 images belonging to 36 classes.


In [ ]:
#Model Initialize
epochs=25
model = classifier.fit_generator(
        training_set,
        steps_per_epoch=7605,	#No of Image in Traing Set 7605
        epochs=epochs, #Previous value is 25
        validation_data = test_set,
        validation_steps =3292 	#No of Image in Test Set 3292
      )

<ipython-input-103-6dd1c3bfab29>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = classifier.fit_generator(


Epoch 1/25
 148/7605 [..............................] - ETA: 22:12:37 - loss: 2.3976 - accuracy: 0.3553

In [ ]:
#Saving the model
classifier.save('/content/drive/MyDrive/Data/Trained_model.h5')

model_json = classifier.to_json()
with open("/content/drive/MyDrive/Data/model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')

print(model.history.keys())

In [ ]:
#Accuracy
model.history['accuracy']